In [230]:
import hashlib
import json
import datetime as dt 

from flask import Flask, jsonify, request
## 분산환경에서 여러 노드가 있을때 개별 node에 대해 식별키를 만들어주기 위해서 사용함
from uuid import uuid4

# bit-coin처럼 transaction 리스트를 모아서 이를 하나의 블록으로 만들어주는 블록체인 클래스를 정의합니다. 
class Blockchain(object):
    def __init__(self):
        ## block이 생성되면 들어가는 리스트입니다. 
        self.chain = []
        ## 아직 새로운 블록으로 만들어지지 못한, transaction들입니다. 
        ## 새로운 블록이 만들어지면 이 리스트가 모두 새로운 블록으로 들어가게 됩니다. 
        ## 또한 개별 transaction은 어떤 형태가 되어야 할까요? json? 
        self.current_transactions = []
        self.create_genesis_block()## 시작하면, genesis_block를 만들어야 함 

    def create_genesis_block(self):
        ## create a genesis block. why its proof is 100?? 
        self.new_block(previous_hash=1, proof=100)
        
    def new_block(self, proof, previous_hash=None):
        # Creates a new Block and adds it to the chain
        ## block에 string, list, ditionary, int 등 매우 기본적인 값들만 저장되어 있는 것을 알 수 있습니다. 
        ## 이는 이 블록체인에서 json으로 serialization하기 때문이죠. 
        ## 만약, 새로운 객체를 만들고, 제가 객체들도 포함된 형태로 블록이 생성될 경우 pickle등 다른 방법으로 serialization하는 것이 좋을 수 있습니다. 
        block = {
            'index': len(self.chain) + 1,
             ## timestamp의 경우 string으로 넣을 수도 있으나, UNIX time으로 넣어야 이후 후처리가 쉬움. 
            'timestamp': dt.datetime.timestamp(dt.datetime.now()),
            'transactions': self.current_transactions.copy(),
            'proof': proof,
            'previous_hash': self.hash(self.chain[-1]) if previous_hash is None else previous_hash, 
        }
        # Reset the current list of transactions
        self.current_transactions = []
        self.chain.append(block)
        return block
    
    def new_transaction(self, sender, recipient, amount):
        # Adds a new transaction to the list of transactions
        # 일단, transaction은 항상 sender, recipient, amount로 표현된다는 것이 정의되어 있습니다. 
        # 각각 str, str, int로 표현되죠. 
        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
        })
        ## 얘는 뭔가, 왜 last_block의 index에 1을 더하여 리턴하는가?? 
        ## 왜 거래가 추가될 block의 index를 리턴하는가?? 
        return self.last_block['index'] + 1
    
    @staticmethod## staticmethod는 그저 클래스 내부에 존재할 뿐이지 외부에 있는 함수처럼 작동합니다(class or class instance의 어떤 정보도 참조하지 않습니다. )
    def hash(block):
        # Hashes a Block
        # 이 블록체인에서는 각각의 block이 json으로 변형되기 쉽도록, dictionary, int, string으로만 구성되어 있습니다. 
        # sort_keys=True를 넘겨주지 않아도 기본적으로 json에서 ordering이 유지됨. 
        block_json = json.dumps(block).encode()
        return hashlib.sha256(block_json).hexdigest()
    
    @property## @property 데코레이터는 해당 함수를 마치 attribute처럼 접근할 수 있게 해줍니다(또한 read-only)
    def last_block(self):
        # Returns the last Block in the chain
        return self.chain[-1]
    
    def proof_of_work(self, last_proof):
        """
        간단한 Proof of work algorithm: 
        - last_proof와 어떤 값(proof)를 합쳐서 넣었을 때, 우리가 원하는 조건을 충족했는지(valid_proof)를 파악하고,
        - 충족할때의 proof를 찾아서 리턴함. 
        """
        proof = 0
        while self.valid_proof(last_proof, proof) is False:
            proof += 1
        return proof
    
    @staticmethod
    def valid_proof(last_proof, proof):
        """
        hash(last_proof, proof)의 제일 앞 네 byte가 0인지를 파악한다. 
        - last_proof는 previous block의 proof고, 
        - 이를 섞은 다음 proof of work를 찾는 놈에게 다음 block를 만들 수 있는 자격이 주어짐. 
        """
        guess = f'{last_proof}{proof}'.encode() ## string ==> binary 
        guess_hash = hashlib.sha256(guess).hexdigest()
        ## 제일 앞 네 개가 0인지 파악함, 물론 보통 시간이 지나면서 이 0의 개수를 늘리는 식으로 진행함. 
        return guess_hash[:4] == "0000"
    
#######################
# flask로 서버를 운영합니다. 
app = Flask(__name__)

# node에 대한 식별자를 무작위로 만들어줍니다. 이때 uuid4를 사용함. 
# mine()에서 이 변수를 사용함. 
node_identifier = str(uuid4()).replace('-', '')

# Instantiate the Blockchain
blockchain = Blockchain()

@app.route('/mine', methods=['GET'])
def mine():
    # mine에 성공하면, 해당 block을 유지하는데 기여했으므로 리워드를 줌
    
    last_block = blockchain.last_block
    last_proof = last_block['proof']
    proof = blockchain.proof_of_work(last_proof)

    # We must receive a reward for finding the proof.
    # The sender is "0" to signify that this node has mined a new coin.
    ## where is identifier?
    blockchain.new_transaction(
        sender="0",
        recipient=node_identifier,
        amount=1,
    )

    # Forge the new Block by adding it to the chain
    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)

    response = {
        'message': "New Block Forged",
        'index': block['index'],
        'transactions': block['transactions'],
        'proof': block['proof'],
        'previous_hash': block['previous_hash'],
    }
    return jsonify(response), 200
  
## 새로운 transaction data를 만들어줍니다. 
@app.route('/transactions/new', methods=['POST'])
def new_transaction():
    values = request.get_json()
    # Check that the required fields are in the POST'ed data
    required = ['sender', 'recipient', 'amount']
    # transcation에는 위 세 가지 정보 sender, recipient, amount가 모두 있어야 함. 
    # 하나라도 없을 경우 에러 발생.
    if not all(k in values for k in required):
        return 'Missing values', 400
    # Create a new Transaction
    index = blockchain.new_transaction(values['sender'], values['recipient'], values['amount'])
    response = {'message': f'Transaction will be added to Block {index}'}
    return jsonify(response), 201

## 현재 전체 blockchain을 리턴해줍니다.
@app.route('/chain', methods=['GET'])
def full_chain():
    response = {
        'chain': blockchain.chain,
        'length': len(blockchain.chain),
    }
    return jsonify(response), 200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

UnsupportedOperation: not writable

In [229]:
import uuid
## make a random UUID
print(uuid.uuid4())

633db812-2df2-438d-b126-7da2eec680c4


In [275]:
from urllib.parse import urlparse
## 아무 url이나 한번 넣어봅시다. 
a = urlparse("https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=urllib")
a = urlparse('https://localhost:5000/mine')

print(a)
print(a.scheme)
print(a.netloc)
print(a.path)
print(a.params)
print(a.query)
print(a.port)
print(a.geturl())



ParseResult(scheme='https', netloc='localhost:5000', path='/mine', params='', query='', fragment='')
https
localhost:5000
/mine


5000
https://localhost:5000/mine


In [294]:
from urllib.parse import urlparse

test_url_lst = [
    "https://docs.python.org/3/library/urllib.parse.html", 
    "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=urllib", 
    "http://localhost:5000/mine"
]

for test_url in test_url_lst:
    x = urlparse(test_url)
    print(f"url: {x.geturl()}")
    print(f"scheme: {x.scheme}")
    print(f"netloc: {x.netloc}")
    print(f"relative path: {x.path}")## relative path 
    print(f"params: {x.params}")
    print(f"query: {x.query}")
    print(f"port: {x.port}")
    print("="*50)

url: https://docs.python.org/3/library/urllib.parse.html
scheme: https
netloc: docs.python.org
relative path: /3/library/urllib.parse.html
params: 
query: 
port: None
url: https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=urllib
scheme: https
netloc: search.naver.com
relative path: /search.naver
params: 
query: where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=urllib
port: None
url: http://localhost:5000/mine
scheme: http
netloc: localhost:5000
relative path: /mine
params: 
query: 
port: 5000


In [274]:
a

In [ ]:
5000 "e662143083214b28b79b33cf666e8c84"
5001 "3c5c8cbdd78d45648a1eb1fd7fa4e8fd"
5002 "c45f461377a14cedb820c69a0d0638ee"